In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as NUM
import arcpy as ARCPY
import arcpy.da as DA
import pandas as PD
import seaborn as SEA
import matplotlib.pyplot as PLOT
import pandas as PD
import arcgisscripting as ARC
import arcpy as ARCPY
import SSUtilities as UTILS
import os as OS

In [11]:
#Names of Prediction Variables#Import USA Seagrass Data 
USAurl = r'https://services3.arcgis.com/oZfKvdlWHN1MwS48/arcgis/rest/services/MachineLearningSeagrass/FeatureServer/0?token=W4CSlYAQxBZKNONRrS_MVm0qNa7j21xD5BNMByQOuXTQ6xhXxFydToukAXYrndK0RRrMNjvbFHcbCOioIVNAgIe6f0iV7rclZ4kNGqoI3i1Ldjhv_UgwzeBGc7KJacuAshcwYXkb6PIgrGipbwD3YOV3AGhhRmh9e5RzMSWB7HJS9fSYzU2l8ZL9vkLmWccKvo_DGJaqm9GIkz9HoPM2hv2yGDdf9SEtcoQvBsNkCWk.'
predictVars = ['salinity', 'temp', 'phosphate','nitrate', 
               'silicate', 'dissO2']
#Name of Classification Variable
classVar = ['Present']
#List of all Variables
allVars = predictVars + classVar
#Create a Data Object
USAlayer = DA.FeatureClassToNumPyArray(USAurl, ["SHAPE@XY"] + allVars)
#Obtain Spatial Reference
spatRefGlobal = ARCPY.Describe(globalUrl).spatialReference
#Define Main Dataframe
USA_Train = PD.DataFrame(USAlayer, columns = allVars)
#Display Portion of the Data Frame
USA_Train.head()

,salinity,temp,phosphate,nitrate,silicate,dissO2,Present
0,33.757988,8.378799,2.04564,25.473619,35.493446,3.365844,0
1,33.757988,8.378799,2.04564,25.473619,35.493446,3.365844,0
2,33.757988,8.378799,2.04564,25.473619,35.493446,3.365844,0
3,33.757988,8.378799,2.04564,25.473619,35.493446,3.365844,0
4,33.757988,8.378799,2.04564,25.473619,35.493446,3.365844,0


In [12]:
##Train Random Forest Using USA Data
#Encode Seagrass Presence as Classes
indicatorUSA, _ = PD.factorize(USA_Train['Present'])
#Create Random Forest Classification Object
rfco = RandomForestClassifier(n_estimators = 500)
#Perform Classification Using Training Set
rfco.fit(USA_Train[predictVars[:-1]], indicatorUSA)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [13]:
## Import Global Data for Prediction ##
globalUrl = r'https://services3.arcgis.com/oZfKvdlWHN1MwS48/arcgis/rest/services/MachineLearningSeagrass/FeatureServer/2?token=W4CSlYAQxBZKNONRrS_MVm0qNa7j21xD5BNMByQOuXTQ6xhXxFydToukAXYrndK0RRrMNjvbFHcbCOioIVNAgIe6f0iV7rclZ4kNGqoI3i1Ldjhv_UgwzeBGc7KJacuAshcwYXkb6PIgrGipbwD3YOV3AGhhRmh9e5RzMSWB7HJS9fSYzU2l8ZL9vkLmWccKvo_DGJaqm9GIkz9HoPM2hv2yGDdf9SEtcoQvBsNkCWk.'
#Create a SS Data Object
globalData = DA.FeatureClassToNumPyArray(globalUrl, ["SHAPE@XY"] + predictVars[:-1])
#Obtain Spatial Reference
spatRefGlobal = ARCPY.Describe(globalUrl).spatialReference
#Define Dataframe
globalTrain = PD.DataFrame(globalData, columns = predictVars[:-1])

In [ ]:
#Predict Global Seagrass Occurance
seagrassPredGlobal = rfco.predict(globalTrain)
#Bring Output Back to ArcGIS
nameFC = 'GlobSeagrass'
outputDir = r'C:\Users\orhu8849\Documents\ArcGIS\Projects\UC_2017_ML_Seminar\UC_2017_ML_Seminar.gdb'
grassExists = globalData[["SHAPE@XY"]][globalTrain.index[NUM.where(seagrassPredGlobal==1)]]
# Write Locations with Seagrass to Feature Class
ARCPY.da.NumPyArrayToFeatureClass(grassExists, OS.path.join(outputDir, nameFC), ['SHAPE@XY'], spatRefGlobal)
#Calculate Point Kernel Density for Predicted Seagrass Locations
ARCPY.sa.KernelDensity(OS.path.join(outputDir, nameFC), "NONE", 104414.5785052, None, "SQUARE_KILOMETERS", "DENSITIES", "PLANAR"); out_raster.save(OS.path.join(outputDir,'globSeagrassDens'))